# [Combine datasets](#combine-datasets)

In [ ]:
%load_ext nb_black
%load_ext autoreload
%autoreload 2

In [ ]:
from datetime import datetime
from glob import glob
from pathlib import Path
from time import time

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
%aimport src.visualization_helpers
from src.visualization_helpers import plot_horiz_bar

%aimport src.preprocess_helpers
from src.preprocess_helpers import (
    append_clean_data,
    append_demographic_data,
    drop_non_zero_rows,
    explode,
    load_merge_slice_data,
    merge_with_weather_data,
    point_inside_polygon,
    write_data_to_csv,
)

In [ ]:
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)

<a id="toc"></a>

## [Table of Contents](#table-of-contents)
0. [About](#about)
1. [User Inputs](#user-inputs)
2. [Load crime listings data](#load-crime-listings-data)
3. [Get demographic data](#get-demographic-data)
4. [Load weather data](#load-weather-data)
5. [Extract neighborhood name from joined data](#extract-neighborhood-name-from-joined-data)
6. [Summarize joined data](#summarize-joined-data)
7. [Export joined data](#export-joined-data)

<a id="about"></a>

## 0. [About](#about)

In this notebook, we will combine crime listings, weather, demographics and GIS data into a single file at `data/all_joined__<YYYYmmdd_HHMMSS>.csv`

<a id="user-inputs"></a>

## 1. [User Inputs](#user-inputs)

We'll define below the variables that are to be used throughout the code.

In [ ]:
data_dir_path = str(Path().cwd() / "data")
weather_data_file_path = str(Path(data_dir_path) / "raw" / "1914019.csv")
joined_data_path = str(
    Path(data_dir_path)
    / "processed"
    / f"all_joined__{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
)
years_wanted = [2018, 2019]
months_wanted = [1, 2, 3]
cols_to_drop = ["ID"]

dtypes_dict = {
    "id": int,
    "case_number": str,
    "date": str,
    "block": str,
    "iucr": str,
    "primary_type": str,
    "description": str,
    "location_description": str,
    "arrest": bool,
    "domestic": bool,
    "beat": int,
    "district": int,
    "ward": float,
    "community_area": float,
    "fbi_code": str,
    "X Coordinate": float,
    "Y Coordinate": float,
    "year": int,
    "updated_on": str,
    "latitude": float,
    "longitude": float,
    "location": str,
    "Historical Wards 2003-2015": float,
    "Zip Codes": float,
    "Community Areas": float,
    "Census Tracts": float,
    "Wards": float,
    "Boundaries - ZIP Codes": float,
    "Police Districts": float,
    "Police Beats": float,
}
unwanted_cols = ["case_number", "date", "x_coordinate", "y_coordinate", "updated_on"]
wanted_weather_cols = [
    "AWND",
    "PRCP",
    "SNOW",
    "SNWD",
    "TAVG",
    "TMAX",
    "TMIN",
    "WDF2",
    "WDF5",
    "WSF2",
    "WSF5",
    "WT01",
    "date_yymmdd",
]

weather_data_date_col = "DATE"
merge_data_on = "date_yymmdd"
merge_weather_data_on = "date_yymmdd"

In [ ]:
data_dir_path, weather_data_file_path, joined_data_path = [
    Path(data_dir_path),
    Path(weather_data_file_path),
    Path(joined_data_path),
]

<a id="load-crime-listings-data"></a>

## 1. [Load crime listings data](#load-crime-listings-data)

We will begin by loading and cleaning the crime listings data from the [Chicago open data portal](https://data.cityofchicago.org/browse?limitTo=datasets) for the years [2018](https://data.cityofchicago.org/Public-Safety/Crimes-2018/3i3m-jwuy) and [2019](https://data.cityofchicago.org/Public-Safety/Crimes-2019/w98m-zvie).

We'll start by performing the following
- read all data files corresponding to crime listings data into a single `DataFrame`
- change all column names to lower case
- replace spaces in column names by underscores
- drop the `ID` column

Next, we'll filter by year(s) and month(s) that we want to include in our analysis - 2018-2019 and January-March respectively

In [ ]:
cell_st = time()

df = load_merge_slice_data(
    dtypes_dict=dtypes_dict,
    file_paths=glob(str(data_dir_path / "raw" / "Crime*.csv")),
    years_wanted=years_wanted,
    months_wanted=months_wanted,
    cols_to_drop=cols_to_drop,
)

total_minutes, total_seconds = divmod(time() - cell_st, 60)
print(
    f"Cell exection time: {int(total_minutes):d} minutes, {total_seconds:.2f} seconds"
)

In [ ]:
print(df.shape[0])
display(df.head())
display(df.dtypes.to_frame())
print(
    f"Number of rows of data left after dropping records with unwanted years and months: {df.shape[0]}"
)

We'll print the number of rows of missing values for all columns of the crime listings data

In [ ]:
df.isna().sum()

While several features have missing data, these include some of the unwanted features we specified in the user input section earlier. Those unwanted columns should be dropped before dropping rows with missing values in any feature.

Now, we'll use a helper fucntion to geneate features based on `datetime` attributes, filter out unwanted columns and then drop rows with any missing values. This will include two boolean features
- `is_weekend` a boolean to check if the date falls on a weekday or weekend
- `is_dark` a boolean to check if the hour of the day falls during a manually specified time of day when the sky is dark outside or not

In [ ]:
df = append_clean_data(df=df, unwanted_cols=unwanted_cols)
df.head(2)

Finally, we'll examine the number of rows of missing data in all columns of the crime listings `DataFrame`

In [ ]:
pd.DataFrame(df.isna().sum(), columns=["number_of_missing_values"])

<a id="get-demographic-data"></a>

## 3. [Get demographic data](#get-demographic-data)

In this section, we'll use the []() package to extract demographic data about the area immediately surrounding the locations where crimes were committed.

In order to do this, we'll start by creating several helper functions to extract a single attribute of demographics data, based on its latitude and longitude
- if demographic data is not available for a specific point, we'll return a zero for that attribute

Next, we'll use a [vectorized version of each function](https://stackoverflow.com/a/52674448/4057186) to retrieve their corresponding demographic attribute based on the latitude and longitude, and append this value to the `DataFrame` of crime data

In [ ]:
cell_st = time()

df_execution_times, d = append_demographic_data(df)

total_minutes, total_seconds = divmod(time() - cell_st, 60)
print(
    f"Cell exection time: {int(total_minutes):d} minutes, {total_seconds:.2f} seconds"
)

Below, we'll show the execution time required to retrieve each demographic attribute

In [ ]:
df_execution_times

Next, we'll check how many rows of the `DataFrame` are missing demographic data
- since the helper functions earlier returned 0 if demographic data was missing, we'll filter for zeros as our proxy for missing demographic data

In [ ]:
df_no_demographic_data = pd.DataFrame(
    df[list(d.keys())][df[list(d.keys())] == 0].count(), columns=["num_rows_of_zeros"]
)
df_no_demographic_data["num_rows"] = df.shape[0]
df_no_demographic_data["num_rows_of_non_zeros"] = (
    df_no_demographic_data["num_rows"] - df_no_demographic_data["num_rows_of_zeros"]
)
df_no_demographic_data["percent_of_zeros"] = (
    df_no_demographic_data["num_rows_of_zeros"] / df_no_demographic_data["num_rows"]
) * 100
df_no_demographic_data

Finally, we'll drop all rows of the joined data where demographic data are recorded as zeros

In [ ]:
df = drop_non_zero_rows(df, col_rows_to_drop="total_population")
print(
    f"Number of rows of data left after dropping records with no demographic data: {df.shape[0]}"
)
df.head()

<a id="load-weather-data"></a>

## 4. [Load weather data](#load-weather-data)

In this section we'll load weather data for the city of Chicago (taken from the [Chicago O'Hare International airport station](https://www.ncdc.noaa.gov/cdo-web/datasets/GHCND/stations/GHCND:USW00094846/detail)) to the crime listings data

We'll start by loading the NOAA weather data and then append a column that gives the `date` as a `datetime`. Next, we'll merge weather and crime data on the common column of `date_yymmdd`, which is the `datetime` version of the raw `date` from each `DataFrame`. As we're only retaining a subset of features of the weather data, we'll only include those when merging weather and crime `DataFrame`s

In [ ]:
df = merge_with_weather_data(
    df_data=df,
    weather_data_file_path=Path(weather_data_file_path),
    weather_data_date_col=weather_data_date_col,
    wanted_weather_cols=wanted_weather_cols,
    merge_data_on=merge_data_on,
    merge_weather_data_on=merge_weather_data_on,
)
df.head(3)

<a id="extract-neighborhood-name-from-joined-data"></a>

## 5. [Extract neighborhood name from joined data](#extract-neighborhood-name-from-joined-data)

Here, we'll append a `neighborhood` column to the joined `DataFrame`.

We'll start by reading in the [Chicago open data boundary file for Neighborhoods](https://data.cityofchicago.org/Facilities-Geographic-Boundaries/Boundaries-Neighborhoods/bbvz-uum9) using [`geopandas`](http://geopandas.org/).

Next, we'll explode all [`POLYGON`](https://macwright.org/2015/03/23/geojson-second-bite.html#polygons) objects (tuples of latitude-longitude co-ordinates) in the `geometry` column of the [`geopandas` `geoDataFrame`](https://gist.github.com/mhweber/cf36bb4e09df9deee5eb54dc6be74d26#gistcomment-2353309) and then store them in a list of latitude-longitude co-ordinates. We're doing this so that we can later [check if a single latitude-longitude tuple is in any of the nested (exploded) list of latitude-longitude tuples](https://medium.com/dataexplorations/working-with-open-data-shape-files-using-geopandas-how-to-match-up-your-data-with-the-areas-9377471e49f2), which we will use to determine the name of the neighborhood where a crime was committed.

In [ ]:
cell_st = time()

gdf_out = explode(
    path_to_file=glob(str(data_dir_path / "raw" / "Neighborhoods" / "*.shp"))[0]
)
display(gdf_out.head(2))

total_minutes, total_seconds = divmod(time() - cell_st, 60)
print(
    f"Cell exection time: {int(total_minutes):d} minutes, {total_seconds:.2f} seconds"
)

Finally, we'll extract the `neighborhood` for each row of the data and append this as a new column to the `DataFrame`

In [ ]:
def get_neighbourhood(row: pd.Series) -> str:
    """
    Get neighborhood from lat, long and neighbourhood boudnary lat-long
    SOURCE: https://medium.com/dataexplorations/
            working-with-open-data-shape-files-using-geopandas-how-to-match-
            up-your-data-with-the-areas-9377471e49f2
    """
    for ix, area in gdf_out.iterrows():
        is_in_area = False
        if row["latitude"] and row["longitude"]:
            is_in_area = point_inside_polygon(
                row["latitude"], row["longitude"], area["geomlist"]
            )
            if is_in_area:
                return area["pri_neigh_x"]
    return ""

In [ ]:
cell_st = time()

df["neighbourhood"] = df.apply(lambda row: get_neighbourhood(row), axis=1)
display(df.head())

total_minutes, total_seconds = divmod(time() - cell_st, 60)
print(
    f"Cell exection time: {int(total_minutes):d} minutes, {total_seconds:.2f} seconds"
)

<a id="summarize-joined-data"></a>

## 6. [Summarize joined data](#summarize-joined-data)

Next, we'll create a `DataFrame` showing all the features that will be written to a `*.csv` file
- `block` will not be written to an output file since this feature is comprised of very low frequency values
- `WT01` will not be written since it is nearly unchanged for all selected rows (winter season months)

In [ ]:
f = {
    # "block": [str, "Name of block where crime occurred", "CATEGORICAL"],
    "arrest": [bool, "Arrest made?", "CATEGORICAL"],
    "domestic": [bool, "Domestic Violence?", "CATEGORICAL"],
    "beat": [int, "Smallest police grographic area", "CATEGORICAL"],
    "district": [int, "Police District", "CATEGORICAL"],
    "ward": [float, "City Council District", "CATEGORICAL"],
    "community_area": [float, "Community identifier", "CATEGORICAL"],
    "fbi_code": [str, "FBI Crime Classification", "CATEGORICAL"],
    "day_name": [str, "weekday", "CATEGORICAL"],
    "month": [int, "month", "CATEGORICAL"],
    "day": [int, "day of month", "CATEGORICAL"],
    "hour": [int, "hour of day", "CATEGORICAL"],
    "weekofyear": [int, "week of year", "CATEGORICAL"],
    "total_population": [int, "Surrounding total population", "NUMERIC"],
    "housing_units": [int, "Number of surrounding housing units", "NUMERIC"],
    "median_household_value": [int, "Median surrounding household value", "NUMERIC"],
    "median_household_income": [int, "Median surrounding household income", "NUMERIC"],
    "occupied_housing_values": [
        int,
        "Number of surrounding occupied housing units",
        "NUMERIC",
    ],
    "TAVG": [float, "Average temperature", "NUMERIC"],
    "SNOW": [float, "Quantity of snowfall", "NUMERIC"],
    # "WT01": [float, "Fog, ice fog, or freezing fog (may include heavy fog)", "CATEGORICAL"],
    "neighbourhood": [str, "Neighborhood containing event", "CATEGORICAL"],
}

In [ ]:
df_cols = pd.DataFrame.from_dict(
    f, orient="index", columns=["dtype", "Description", "Feature_Type"]
)
df_cols = df_cols.merge(
    pd.DataFrame(df[list(f.keys())].nunique(), columns=["nunique"]),
    left_index=True,
    right_index=True,
)
df_cols.loc[df_cols["Feature_Type"] == "NUMERIC", "nunique"] = np.nan
df_cols

Finally, we'll show a bar chart of the `primary_type` column (i.e. the type of crime) in order ot visualize class imbalance

In [ ]:
plot_horiz_bar(
    df=df,
    col_name="primary_type",
    ptitle=f"Class Balance for {df.shape[0]:,d} instances",
    fig_size=(10, 10),
    savefig=Path().cwd() / "reports" / "figures",
)

<a id="export-joined-data"></a>

## 7. [Export joined data](#export-joined-data)

In [ ]:
write_data_to_csv(df=df, joined_data_path=joined_data_path, write_index=True)